In [37]:
import lightning as L
import torch
import torch.nn.functional as F
import torchmetrics
from lightning.pytorch.loggers import CSVLogger

from tools import CustomDataModule

In [38]:
class PyTorchMLP(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super().__init__()

        self.all_layers = torch.nn.Sequential(
            # 1st hidden layer
            torch.nn.Linear(num_features, 100),
            torch.nn.BatchNorm1d(100),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(100, 50),
            torch.nn.BatchNorm1d(50),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(50, num_classes),
        )

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        logits = self.all_layers(x)
        return logits

In [39]:
class LightningModel(L.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()

        self.example_input_array = torch.Tensor(5, 100)  # batch_size=5, features=100
        self.learning_rate = learning_rate
        self.model = model

        self.save_hyperparameters(ignore=["model"])

        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)

    def forward(self, x):
        return self.model(x)

    def _shared_step(self, batch):
        features, true_labels = batch
        logits = self(features)

        loss = F.cross_entropy(logits, true_labels)
        predicted_labels = torch.argmax(logits, dim=1)
        return loss, true_labels, predicted_labels

    def training_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)

        self.log("train_loss", loss)
        self.train_acc(predicted_labels, true_labels)
        self.log(
            "train_acc", self.train_acc, prog_bar=True, on_epoch=True, on_step=False
        )
        return loss

    def validation_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)

        self.log("val_loss", loss, prog_bar=True)
        self.val_acc(predicted_labels, true_labels)
        self.log("val_acc", self.val_acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)
        self.test_acc(predicted_labels, true_labels)
        self.log("test_acc", self.test_acc)

    def configure_optimizers(self):
        opt = torch.optim.SGD(self.parameters(), lr=self.learning_rate)
        return opt

In [40]:
L.seed_everything(23)
d_module = CustomDataModule()

pt_model = PyTorchMLP(num_features=100, num_classes=2)
lt_model = LightningModel(model=pt_model, learning_rate=0.05)


Global seed set to 23


In [41]:
from lightning.pytorch.utilities.model_summary import ModelSummary

summary = ModelSummary(lt_model, max_depth=1)
print(summary)

  | Name      | Type               | Params | In sizes | Out sizes
------------------------------------------------------------------------
0 | model     | PyTorchMLP         | 15.6 K | [5, 100] | [5, 2]   
1 | train_acc | MulticlassAccuracy | 0      | ?        | ?        
2 | val_acc   | MulticlassAccuracy | 0      | ?        | ?        
3 | test_acc  | MulticlassAccuracy | 0      | ?        | ?        
------------------------------------------------------------------------
15.6 K    Trainable params
0         Non-trainable params
15.6 K    Total params
0.062     Total estimated model params size (MB)


In [46]:
trainer = L.Trainer(
    # fast_dev_run=5, # use only 5 minibatches to see if there is an overfitting problem in model
    # overfit_batches=1, # if there is not 1.0 accuracy for training, then there is a problem in model
    max_epochs=10,
    accelerator="mps",
    devices="auto",
    logger=CSVLogger(save_dir="csv_logs", name="lt_debug_model"),
    deterministic=True,
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [47]:
trainer.fit(lt_model, datamodule=d_module)


  | Name      | Type               | Params | In sizes | Out sizes
------------------------------------------------------------------------
0 | model     | PyTorchMLP         | 15.6 K | [5, 100] | [5, 2]   
1 | train_acc | MulticlassAccuracy | 0      | ?        | ?        
2 | val_acc   | MulticlassAccuracy | 0      | ?        | ?        
3 | test_acc  | MulticlassAccuracy | 0      | ?        | ?        
------------------------------------------------------------------------
15.6 K    Trainable params
0         Non-trainable params
15.6 K    Total params
0.062     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [48]:
trainer.test(model=lt_model, datamodule=d_module)

/Users/alexandragridneva/ProgrammingTools/miniconda3/envs/dl_lightning_ai_course/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8602499961853027     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.8602499961853027}]